In [31]:
import numpy as np
import heapq 
from collections import namedtuple
import random
from random import choice
from tqdm import tqdm

np.random.seed(2024)
random.seed(2024)

action = namedtuple('Action', ['pos1', 'pos2'])

PUZZLE_DIM = 4
RANDOMIZE_STEPS = 100_000

In [32]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state



def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int: 
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:   
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)  
                distance += abs(x - goal_x) + abs(y - goal_y)   
    return distance

In [33]:

def a_star(start_state: np.ndarray, target_state: np.ndarray):
    open_list = []
    explored_states = set()
    parent_map = {}
    state_cost = {}
    
    # Convert states to tuples
    start_state_tuple = tuple(start_state.flatten())
    target_state_tuple = tuple(target_state.flatten())
    
    # Heuristic cost (Manhattan distance)
    initial_heuristic = manhattan_distance(start_state, target_state)
    state_cost[start_state_tuple] = 0
    
    # Add the start state to the open list
    heapq.heappush(open_list, (initial_heuristic, start_state_tuple))
    explored_states.add(start_state_tuple)
    
    iteration_count = 0  
    
    while open_list:
        iteration_count += 1

        # Get the state with the lowest f value
        _, current_state = heapq.heappop(open_list)
        
        # Goal check
        if current_state == target_state_tuple:
            solution_path = []
            state = current_state
            while state != start_state_tuple:
                solution_path.append(state)
                state = parent_map[state]
            solution_path.reverse()
            return solution_path, state_cost[current_state], iteration_count
        
        explored_states.add(current_state)
        
        reshaped_state = np.array(current_state).reshape((PUZZLE_DIM, PUZZLE_DIM))
        for move in available_actions(reshaped_state):
            next_state = do_action(reshaped_state, move)
            next_state_tuple = tuple(next_state.flatten())
            
            if not next_state_tuple in explored_states:
                new_g_value = state_cost[current_state] + 1
                new_h_value = manhattan_distance(next_state, target_state)
                new_f_value = new_g_value + new_h_value
                
                # Update if it's a better path
                if next_state_tuple not in state_cost or new_g_value < state_cost[next_state_tuple]:
                    state_cost[next_state_tuple] = new_g_value
                    heapq.heappush(open_list, (new_f_value, next_state_tuple))
                    parent_map[next_state_tuple] = current_state
    
    print("No found")
    return None

In [34]:
 

def bidirectional_a_star(start_state: np.ndarray, target_state: np.ndarray):
    open_list_top = []
    open_list_bottom = []

    explored_states_top = set()
    explored_states_bottom = set()

    state_cost_top = {}
    state_cost_bottom = {}

    # Convert states to tuples
    start_state_tuple = tuple(start_state.flatten())
    target_state_tuple = tuple(target_state.flatten())

    # Heuristic cost (Manhattan distance)
    initial_heuristic = manhattan_distance(start_state, target_state)
    state_cost_top[start_state_tuple] = 0
    state_cost_bottom[target_state_tuple] = 0

    # Add the start state to the open list top and the target to bottom
    heapq.heappush(open_list_top, (initial_heuristic, start_state_tuple))
    heapq.heappush(open_list_bottom, (initial_heuristic, target_state_tuple))

    total_explored_states = 0

    while open_list_top and open_list_bottom:
        # Expand from the top search
        _, current_state_top = heapq.heappop(open_list_top)
        explored_states_top.add(current_state_top)
        total_explored_states += 1

        # Expand from the bottom search
        _, current_state_bottom = heapq.heappop(open_list_bottom)
        explored_states_bottom.add(current_state_bottom)
        total_explored_states += 1

        # Check if the two searches meet
        if current_state_top in explored_states_bottom:
            g_top = state_cost_top[current_state_top]
            g_down = state_cost_bottom[current_state_top]
            return g_top + g_down, total_explored_states

        if current_state_bottom in explored_states_top:
            g_top = state_cost_top[current_state_bottom]
            g_down = state_cost_bottom[current_state_bottom]
            return  g_top + g_down, total_explored_states

        # Expand neighbors for the top search
        expand_neighbors(
            current_state_top, open_list_top, explored_states_top, state_cost_top,
            target_state_tuple, state_cost_bottom, explored_states_bottom
        )

        # Expand neighbors for the bottom search
        expand_neighbors(
            current_state_bottom, open_list_bottom, explored_states_bottom, state_cost_bottom,
            start_state_tuple, state_cost_top, explored_states_top
        )

    print("No solution found.")
    return None

def expand_neighbors(
    current_state, open_list, explored_states, state_cost,
    other_target, other_state_cost, other_explored_states
):
    reshaped_state = np.array(current_state).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for move in available_actions(reshaped_state):
        next_state = do_action(reshaped_state, move)
        next_state_tuple = tuple(next_state.flatten())

        if next_state_tuple not in explored_states:
            new_g_value = state_cost[current_state] + 1
            if next_state_tuple in other_state_cost:
                new_h_value = 0  # If meeting point is found
            else:
                new_h_value = manhattan_distance(next_state, np.array(other_target).reshape((PUZZLE_DIM, PUZZLE_DIM)))

            new_f_value = new_g_value + new_h_value

            # Update if it's a better path
            if next_state_tuple not in state_cost or new_g_value < state_cost[next_state_tuple]:
                state_cost[next_state_tuple] = new_g_value
                heapq.heappush(open_list, (new_f_value, next_state_tuple))



In [35]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
initial_state = goal_state.copy()
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))
 

print("Initial state:")
print(initial_state)

exact_path, steps, iterations = a_star(initial_state, goal_state)
print("A* (exact solution):")
print(f"number of iterations: {iterations}")
print(f"number of steps to solve puzzle: {steps}")


steps, iterations = bidirectional_a_star(initial_state, goal_state)
print("\nA* bidirectional (heuristic solution):")
print(f"number of iterations: {iterations}")
print(f"number of steps to solve puzzle: {steps}")


Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 335775.33it/s]


Initial state:
[[ 3  1  2 14]
 [ 9  5  7  4]
 [12 15  0  8]
 [ 6 11 10 13]]
A* (exact solution):
number of iterations: 1540131
number of steps to solve puzzle: 46

A* bidirectional (heuristic solution):
number of iterations: 56956
number of steps to solve puzzle: 48
